In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
tickets = pd.read_csv('anonymized_hubspot_tickets.csv')
deals = pd.read_csv('anonymized_hubspot_deals.csv')
companies = pd.read_csv('anonymized_hubspot_companies.csv')

C:\Users\zhaos\AppData\Local\Temp\ipykernel_11084\2095908849.py:3: DtypeWarning: Columns (24,25,28,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  companies = pd.read_csv('anonymized_hubspot_companies.csv')


In [ ]:
# --- Company Size Category ---
def size_category(num_employees):
    if pd.isnull(num_employees):
        return np.nan
    elif num_employees < 10:
        return "Very Small"
    elif num_employees < 50:
        return "Small"
    elif num_employees < 250:
        return "Medium"
    elif num_employees < 1000:
        return "Large"
    else:
        return "Enterprise"

companies['Company_Size_Category'] = companies['Number of Employees'].apply(size_category)

# --- Revenue Category ---
def revenue_category(revenue):
    if pd.isnull(revenue):
        return np.nan
    elif revenue < 1_000_000:
        return "<$1M"
    elif revenue < 10_000_000:
        return "$1M-$10M"
    elif revenue < 50_000_000:
        return "$10M-$50M"
    elif revenue < 100_000_000:
        return "$50M-$100M"
    elif revenue < 500_000_000:
        return "$100M-$500M"
    else:
        return ">$500M"

companies['Revenue_Category'] = companies['Annual Revenue'].apply(revenue_category)

# --- Industry Standardized ---
companies['Industry_Standardized'] = companies['Industry'].str.title()

# --- Region ---
companies['Region'] = companies['Country/Region'].str.strip()

# --- Uses_[Technology] flags ---
technology_keywords = ['Google Tag Manager', 'Salesforce', 'Microsoft Office 365']
for tech in technology_keywords:
    col_name = f'Uses_{tech.replace(" ", "_")}'
    companies[col_name] = companies['Web Technologies'].fillna('').apply(lambda x: int(tech.lower() in x.lower()))

# --- Technology Count ---
companies['Technology_Count'] = companies['Web Technologies'].fillna('').apply(lambda x: len([t for t in x.split(';') if t.strip()]))

# --- Is_BPO flag ---
companies['Is_BPO'] = companies['BPO'].str.lower().map({'yes': 1, 'no': 0})

# --- Date-based Fields ---
companies['Create Date'] = pd.to_datetime(companies['Create Date'], errors='coerce')
companies['Create_Year'] = companies['Create Date'].dt.year
companies['Create_Month'] = companies['Create Date'].dt.month
companies['Create_Quarter'] = companies['Create Date'].dt.quarter
companies['Create_YearMonth'] = companies['Create Date'].dt.to_period('M').astype(str)


In [10]:
# Clean up column names just in case
tickets.columns = tickets.columns.str.strip()

# --- Convert date columns (only those that exist) ---
date_cols = [
    'Create date', 'Close date', '1st Syms presented for review',
    '1st syms run in production',  # This column may still not exist; remove if needed
]
for col in date_cols:
    if col in tickets.columns:
        tickets[col] = pd.to_datetime(tickets[col], errors='coerce')

# --- Implementation Duration in Days ---
tickets['Implementation_Duration_Days'] = (
    tickets['Close date'] - tickets['Create date']
).dt.days

# --- Days to First Sym ---
tickets['Days_To_First_Sym'] = (
    tickets['1st Syms presented for review'] - tickets['Create date']
).dt.days

# --- Time to Close in Hours ---
def convert_to_hours(x):
    try:
        h, m, s = map(int, str(x).split(':'))
        return h + m/60 + s/3600
    except:
        return np.nan

tickets['Time_To_Close_Hours'] = tickets['Time to close (HH:mm:ss)'].apply(convert_to_hours)

# --- Implementation Status (simplified) ---
status_map = {
    'New': 'Not Started',
    'In Progress': 'Ongoing',
    'Waiting on contact': 'Ongoing',
    'Waiting on us': 'Ongoing',
    'Closed': 'Completed'
}
tickets['Implementation_Status'] = tickets['Ticket status'].map(status_map)

# --- Training Completion Count & Percentage ---
training_cols = [
    'Training: Sym Building 101',
    'Training: Sym Building 201',
    'Training: General Overview',
    'Training: Reporting',
    'Training: Deployment/User Management Training'
]
tickets['Training_Completion_Count'] = tickets[training_cols].notna().sum(axis=1)
tickets['Training_Completion_Pct'] = (tickets['Training_Completion_Count'] / len(training_cols)) * 100

# --- Create Year, Month, YearMonth ---
tickets['Create_Year'] = tickets['Create date'].dt.year
tickets['Create_Month'] = tickets['Create date'].dt.month
tickets['Create_YearMonth'] = tickets['Create date'].dt.to_period('M').astype(str)


In [12]:
# Strip column names just in case
deals.columns = deals.columns.str.strip()

# --- Parse date columns ---
deals['Create Date'] = pd.to_datetime(deals['Create Date'], errors='coerce')
deals['Close Date'] = pd.to_datetime(deals['Close Date'], errors='coerce')

# --- Create Year/Month/Quarter ---
deals['Create_Year'] = deals['Create Date'].dt.year
deals['Create_Month'] = deals['Create Date'].dt.month
deals['Create_Quarter'] = deals['Create Date'].dt.quarter

deals['Close_Year'] = deals['Close Date'].dt.year
deals['Close_Month'] = deals['Close Date'].dt.month

# --- YearMonth string for time-based grouping ---
deals['YearMonth'] = deals['Create Date'].dt.to_period('M').astype(str)

# --- Deal Size Category ---
def deal_size_category(amount):
    if pd.isnull(amount):
        return None
    elif amount < 10000:
        return "Small"
    elif amount < 50000:
        return "Medium"
    elif amount < 250000:
        return "Large"
    else:
        return "Enterprise"

deals['Deal_Size_Category'] = deals['Amount'].apply(deal_size_category)



In [14]:
# Drop columns with >95% missing
def drop_high_na_columns(df, threshold=0.95):
    return df.loc[:, df.isnull().mean() <= threshold]

companies_cleaned = drop_high_na_columns(companies)
tickets_cleaned = drop_high_na_columns(tickets)
deals_cleaned = drop_high_na_columns(deals)

# Detect outliers using IQR
def detect_outliers(df):
    outlier_info = {}
    for col in df.select_dtypes(include=['number']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)][col]
        if not outliers.empty:
            outlier_info[col] = {
                "lower_bound": lower,
                "upper_bound": upper,
                "num_outliers": len(outliers),
                "outlier_indices": outliers.index.tolist()[:5]
            }
    return outlier_info

# Run outlier detection
companies_outliers = detect_outliers(companies_cleaned)
tickets_outliers = detect_outliers(tickets_cleaned)
deals_outliers = detect_outliers(deals_cleaned)


In [15]:
print(companies_outliers)

{'Annual Revenue': {'lower_bound': -1493131355.0, 'upper_bound': 2495878813.0, 'num_outliers': 2828, 'outlier_indices': [7, 8, 10, 13, 14]}, 'Number of Form Submissions': {'lower_bound': 0.0, 'upper_bound': 0.0, 'num_outliers': 450, 'outlier_indices': [101, 128, 133, 135, 137]}, 'Number of times contacted': {'lower_bound': -19.5, 'upper_bound': 40.5, 'num_outliers': 1421, 'outlier_indices': [19, 20, 23, 38, 47]}, 'Number of Pageviews': {'lower_bound': 0.0, 'upper_bound': 0.0, 'num_outliers': 427, 'outlier_indices': [101, 128, 133, 135, 139]}, 'Year Founded': {'lower_bound': 1872.5, 'upper_bound': 2084.5, 'num_outliers': 804, 'outlier_indices': [2, 26, 28, 43, 45]}, 'Number of Employees': {'lower_bound': -6875.0, 'upper_bound': 12125.0, 'num_outliers': 2196, 'outlier_indices': [10, 11, 13, 15, 18]}, 'Number of Sessions': {'lower_bound': 0.0, 'upper_bound': 0.0, 'num_outliers': 488, 'outlier_indices': [101, 128, 133, 135, 137]}, 'Technology_Count': {'lower_bound': -4.5, 'upper_bound': 23

In [19]:
print(companies_cleaned.columns)

Index(['CCaaS', 'Annual Revenue', 'Associated Contact',
       'Number of Form Submissions', 'Web Technologies',
       'Number of times contacted', 'Contact with Primary Company',
       'ICP Fit Level', 'Record ID', 'Time Zone', 'Primary Industry',
       'Number of Pageviews', 'Year Founded', 'ICP', 'Segmentation',
       'State/Region', 'Consolidated Industry', 'Type', 'Number of Employees',
       'Primary Sub-Industry', 'Number of Sessions', 'Country/Region',
       'Industry', 'Create Date', 'Company name', 'Last Modified Date',
       'Company_Size_Category', 'Revenue_Category', 'Industry_Standardized',
       'Region', 'Uses_Google_Tag_Manager', 'Uses_Salesforce',
       'Uses_Microsoft_Office_365', 'Technology_Count', 'Create_Year',
       'Create_Month', 'Create_Quarter', 'Create_YearMonth'],
      dtype='object')


In [20]:
# Define the relevant columns to keep
columns_to_keep = [
    'Record ID', 'Company name', 'Industry', 'Industry_Standardized', 'Consolidated Industry',
    'Primary Industry', 'Primary Sub-Industry', 'Country/Region', 'Region', 'State/Region', 'Time Zone',
    'Annual Revenue', 'Revenue_Category', 'Number of Employees', 'Company_Size_Category',
    'Number of Form Submissions', 'Number of Pageviews', 'Number of Sessions', 'Number of times contacted',
    'Technology_Count', 'Uses_Google_Tag_Manager', 'Uses_Salesforce', 'Uses_Microsoft_Office_365',
    'Create Date', 'Create_Year', 'Create_Quarter'
]

# Subset the dataframe
companies_segment = companies_cleaned[columns_to_keep]


In [21]:
print(deals_cleaned.columns)

Index(['Weighted amount', 'Deal Description',
       'Cumulative time in "BANT Deal. Pain ID'ed (Sales Pipeline)" (HH:mm:ss)',
       'Cumulative time in "Opportunity (Sales Pipeline)" (HH:mm:ss)',
       'Days to close', 'Deal Score', 'Close Date',
       'Deal source attribution 2',
       'Cumulative time in "In Trial - Trial in Progress (Sales Pipeline)" (HH:mm:ss)',
       'Contract Start Date',
       'Cumulative time in "Partner Referrals  (Sales Pipeline)" (HH:mm:ss)',
       'Cumulative time in "Deep Dive. PSP Drafted (Sales Pipeline)" (HH:mm:ss)',
       'Pipeline', 'Record ID', 'Forecast category', 'Original Traffic Source',
       'Associated Company', 'Deal owner', 'Is Closed (numeric)',
       'Amount in company currency', 'Deal probability',
       'Associated Company (Primary)', 'Is Closed Won', 'Contract End Date',
       'Last Activity Date', 'Contract Term (Months)', 'Trial Start date',
       'Is closed lost', 'Weighted amount in company currency',
       'Is Deal C

In [22]:
# Define relevant columns for segmentation and deal prediction
columns_to_keep = [
    'Record ID', 'Deal Name', 'Associated Company', 'Associated Company (Primary)',
    'Amount', 'Weighted amount', 'Forecast amount', 'Amount in company currency',
    'Deal Score', 'Deal probability', 'Forecast category', 'Pipeline', 'Deal Stage',
    'Deal Type', 'Is Closed (numeric)', 'Is closed lost', 'Is Closed Won', 'Is Deal Closed?',
    'Is Open (numeric)', 'Contract Term (Months)', 'Days to close', 'Create Date',
    'Close Date', 'Create_Year', 'Create_Quarter', 'Close_Year', 'Deal_Size_Category',
    'Original Traffic Source', 'Deal source attribution 2'
]

# Subset the deals dataframe
deals_segment = deals_cleaned[columns_to_keep]


In [23]:
print(tickets_cleaned.columns)

Index(['Create date', 'Stage Date - Project Launch', 'Associated Contact',
       'Target Launch Date', 'Kickoff Call', 'Close date', 'Pipeline',
       'Stage Date - Project Initiation', 'Ticket status',
       'Stage Date - Execution', 'Time to first agent email reply (HH:mm:ss)',
       'Associated Company', 'Stage Date - Closure Phase',
       'Associated Company (Primary)', 'Response time (HH:mm:ss)', 'Ticket ID',
       'Time to close (HH:mm:ss)', 'Associated Deal', 'Library index approved',
       'Training: General Overview', 'Latest Milestone',
       'Training: Deployment/User Management Training',
       'Requirements for the Trial', 'Training: Sym Building 101',
       'Last modified date', 'Trial End Date', 'Training: Sym Building 201',
       'Stage Date - Converted Won', 'Latest Milestone Update Date',
       'Ticket name', 'Trial Overview', 'Trial Start Date',
       '1st Syms presented for review', 'Project Launch Day',
       'Training: Reporting', 'Construction of 1s

In [24]:
# Define relevant columns for segmentation and prediction
columns_to_keep = [
    'Ticket ID', 'Ticket name', 'Associated Company', 'Associated Company (Primary)', 'Associated Deal',
    'Create date', 'Close date', 'Implementation_Duration_Days', 'Days_To_First_Sym', 'Time_To_Close_Hours',
    'Create_Year', 'Create_Month', 'Create_YearMonth',
    'Ticket status', 'Pipeline', 'Latest Milestone',
    'Trial Start Date', 'Trial End Date',
    'Training_Completion_Count', 'Training_Completion_Pct',
    'Time to first agent email reply (HH:mm:ss)', 'Response time (HH:mm:ss)'
]

# Subset the tickets dataframe
tickets_segment = tickets_cleaned[columns_to_keep]


### Data cleaning for companies

In [ ]:
# Further data cleaning on companies
# 1. Drop rows with missing key segmentation fields
companies_cleaned = companies_cleaned.dropna(subset=['Industry', 'Annual Revenue', 'Number of Employees'])

# 2. Standardize Country/Region
companies_cleaned['Country/Region'] = companies_cleaned['Country/Region'].astype(str).str.strip().str.title()

# 3. Clean Web Technologies field
def clean_web_tech(text):
    if pd.isna(text):
        return []
    return [tech.strip() for tech in text.split(';') if tech.strip()]

companies_cleaned['Web Technologies'] = companies_cleaned['Web Technologies'].apply(clean_web_tech)

# 4. Drop parent/child company relationship fields if present and mostly empty
cols_to_drop = ['Parent Company', 'Associated Company']
companies_cleaned = companies_cleaned.drop(columns=[col for col in cols_to_drop if col in companies_cleaned.columns])


### Data cleaning for tickets

In [ ]:
# 1. Convert time columns to seconds
def time_to_seconds(x):
    try:
        h, m, s = map(int, str(x).split(":"))
        return h * 3600 + m * 60 + s
    except:
        return np.nan

time_columns = ['Time to close (HH:mm:ss)', 'Response time (HH:mm:ss)', 'Time to first agent email reply (HH:mm:ss)']
for col in time_columns:
    if col in tickets_cleaned.columns:
        tickets_cleaned[col + '_seconds'] = tickets_cleaned[col].apply(time_to_seconds)

# 2. Training field completeness (flag if at least one completed)
training_cols = [
    'Training: Sym Building 101',
    'Training: Sym Building 201',
    'Training: General Overview',
    'Training: Reporting',
    'Training: Deployment/User Management Training'
]
for col in training_cols:
    if col in tickets_cleaned.columns:
        tickets_cleaned[col + '_Completed'] = tickets_cleaned[col].notna().astype(int)

# 3. Flag for ongoing implementation (missing milestone allowed)
tickets_cleaned['Is_Closed'] = tickets_cleaned['Ticket status'].str.lower() == 'closed'

# 4. Flag for whether ticket is linked to a deal
tickets_cleaned['Has_Associated_Deal'] = tickets_cleaned['Associated Deal'].notna().astype(int)


### Data cleaning for deals

In [26]:
# 1. Handle missing Amount values
# Option 1: Drop rows where 'Amount' is missing
deals_cleaned = deals_cleaned[deals_cleaned['Amount'].notna()]

# Option 2 (alternative): Impute with median by Deal Type
# deals_cleaned['Amount'] = deals_cleaned.groupby('Deal Type')['Amount'].transform(lambda x: x.fillna(x.median()))

# 2. Remove outliers in Days to close using IQR
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

if 'Days to close' in deals_cleaned.columns:
    deals_cleaned = remove_outliers_iqr(deals_cleaned, 'Days to close')

# 3. Optional: Bin Deal probability into categories
def bin_probability(p):
    if pd.isna(p):
        return 'Unknown'
    elif p < 0.3:
        return 'Low'
    elif p < 0.7:
        return 'Medium'
    else:
        return 'High'

deals_cleaned['Deal_Probability_Bin'] = deals_cleaned['Deal probability'].apply(bin_probability)


In [28]:
# Ensure deal identifier fields are strings
tickets_cleaned.loc[:, 'Associated Deal'] = tickets_cleaned['Associated Deal'].astype(str)
deals_cleaned.loc[:, 'Deal Name'] = deals_cleaned['Deal Name'].astype(str)

# Find tickets whose 'Associated Deal' matches a 'Deal Name'
overlapping_deals = tickets_cleaned[tickets_cleaned['Associated Deal'].isin(deals_cleaned['Deal Name'])]

# Summary stats
overlap_count = overlapping_deals['Associated Deal'].nunique()
total_tickets_with_deals = tickets_cleaned['Associated Deal'].notnull().sum()

print("Total tickets with non-null 'Associated Deal':", total_tickets_with_deals)
print("Matching deals found in deals_cleaned dataset:", overlap_count)


Total tickets with non-null 'Associated Deal': 79
Matching deals found in deals_cleaned dataset: 27


### Merge tickets and deals

In [31]:
import json

# Step 1: Load mapping file (CompanyToTickets)
with open('mappings.json', 'r') as f:
    mappings = json.load(f)

# Load mappings.json
with open("mappings.json", "r") as f:
    mappings = json.load(f)

# Extract TicketToDeal mapping
ticket_to_deal = mappings["TicketToDeal"]

# Map ticket IDs to Deal Names
tickets_cleaned.loc[:, "Ticket ID"] = tickets_cleaned["Ticket ID"].astype(str)
deals_cleaned.loc[:, "Deal Name"] = deals_cleaned["Deal Name"].astype(str)
tickets_cleaned.loc[:, "Mapped Deal Name"] = tickets_cleaned["Ticket ID"].map(ticket_to_deal)

# Merge using Deal Name (not Record ID!)
merged_tickets_deals = pd.merge(
    tickets_cleaned,
    deals_cleaned,
    left_on="Mapped Deal Name",
    right_on="Deal Name",
    how="inner",
    suffixes=("_ticket", "_deal")
)

# Optional: Summary of the merge
merge_report = {
    "Tickets with mapped deal": tickets_cleaned["Mapped Deal Name"].notnull().sum(),
    "Merged Rows": merged_tickets_deals.shape[0],
    "Unique Ticket IDs in Merge": merged_tickets_deals["Ticket ID"].nunique(),
    "Unique Deal Names in Merge": merged_tickets_deals["Deal Name"].nunique()
}

print("✅ Merge Summary:")
for k, v in merge_report.items():
    print(f"{k}: {v}")

✅ Merge Summary:
Tickets with mapped deal: 58
Merged Rows: 43
Unique Ticket IDs in Merge: 42
Unique Deal Names in Merge: 27


In [32]:
# Check the merge quality
# Check total number of tickets that had a mapping
total_mapped_tickets = tickets_cleaned["Mapped Deal Name"].notnull().sum()

# Check how many tickets successfully merged
total_merged_rows = merged_tickets_deals.shape[0]
unique_ticket_ids = merged_tickets_deals["Ticket ID"].nunique()
unique_deal_names = merged_tickets_deals["Deal Name"].nunique()

# Check for duplicates (1 ticket → multiple deal rows)
duplicate_ticket_ids = merged_tickets_deals["Ticket ID"].duplicated().sum()

# Check for deals associated with multiple tickets
deal_to_ticket_counts = merged_tickets_deals["Mapped Deal Name"].value_counts()
deals_with_multiple_tickets = (deal_to_ticket_counts > 1).sum()

# Optional: Check how many merged deals are Closed Won
if "Is Closed Won" in merged_tickets_deals.columns:
    closed_won_count = merged_tickets_deals["Is Closed Won"].sum()
else:
    closed_won_count = "Not Available"

# Compile report
merge_quality_report = {
    "Tickets with mapped deals": total_mapped_tickets,
    "Merged rows": total_merged_rows,
    "Unique ticket IDs in merged data": unique_ticket_ids,
    "Unique deal names in merged data": unique_deal_names,
    "Duplicate ticket IDs in merged data": duplicate_ticket_ids,
    "Deals linked to multiple tickets": deals_with_multiple_tickets,
    "Closed Won deals (if available)": closed_won_count
}

# Display report
print("📊 Merge Quality Report")
for key, value in merge_quality_report.items():
    print(f"{key}: {value}")


📊 Merge Quality Report
Tickets with mapped deals: 58
Merged rows: 43
Unique ticket IDs in merged data: 42
Unique deal names in merged data: 27
Duplicate ticket IDs in merged data: 1
Deals linked to multiple tickets: 9
Closed Won deals (if available): 26


In [36]:
# Step 1: Ensure keys are strings and merge
merged_tickets_deals['Associated Company (Primary)_deal'] = merged_tickets_deals['Associated Company (Primary)_deal'].astype(str).str.strip()
companies_cleaned['Company name'] = companies_cleaned['Company name'].astype(str).str.strip()

merged_full = pd.merge(
    merged_tickets_deals,
    companies_cleaned,
    left_on='Associated Company (Primary)_deal',
    right_on='Company name',
    how='left'
)

# Step 2: Select relevant features for segmentation and value analysis
segmentation_cols = [
    'Company name', 'Industry_Standardized', 'Region', 'Revenue_Category',
    'Company_Size_Category', 'Technology_Count', 'Uses_Google_Tag_Manager',
    'Uses_Salesforce', 'Uses_Microsoft_Office_365', 'Annual Revenue',
    'Number of Employees', 'Amount', 'Forecast amount', 'Deal Score',
    'Deal probability', 'Pipeline_ticket', 'Ticket status', 'Training_Completion_Pct',
    'Implementation_Duration_Days', 'Time_To_Close_Hours'
]

# Step 3: Clean the subset
segmentation_data = merged_full[segmentation_cols].dropna()

# Step 4: Optional — convert categorical vars to category dtype (for modeling or clustering)
categorical_cols = [
    'Industry_Standardized', 'Region', 'Revenue_Category',
    'Company_Size_Category', 'Pipeline_ticket', 'Ticket status'
]
for col in categorical_cols:
    segmentation_data[col] = segmentation_data[col].astype('category')


In [34]:
print(merged_tickets_deals.columns.tolist())


['Create date', 'Stage Date - Project Launch', 'Associated Contact', 'Target Launch Date', 'Kickoff Call', 'Close date', 'Pipeline_ticket', 'Stage Date - Project Initiation', 'Ticket status', 'Stage Date - Execution', 'Time to first agent email reply (HH:mm:ss)', 'Associated Company_ticket', 'Stage Date - Closure Phase', 'Associated Company (Primary)_ticket', 'Response time (HH:mm:ss)', 'Ticket ID', 'Time to close (HH:mm:ss)', 'Associated Deal', 'Library index approved', 'Training: General Overview', 'Latest Milestone', 'Training: Deployment/User Management Training', 'Requirements for the Trial', 'Training: Sym Building 101', 'Last modified date', 'Trial End Date_ticket', 'Training: Sym Building 201', 'Stage Date - Converted Won', 'Latest Milestone Update Date', 'Ticket name', 'Trial Overview', 'Trial Start Date', '1st Syms presented for review', 'Project Launch Day', 'Training: Reporting', 'Construction of 1st Sym begun', 'Implementation_Duration_Days', 'Days_To_First_Sym', 'Time_To_

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# STEP 1: Merge (adjust these to match your variable names)
merged_tickets_deals['Associated Company (Primary)_deal'] = merged_tickets_deals['Associated Company (Primary)_deal'].astype(str).str.strip()
companies_cleaned['Company name'] = companies_cleaned['Company name'].astype(str).str.strip()

merged_full = pd.merge(
    merged_tickets_deals,
    companies_cleaned,
    left_on='Associated Company (Primary)_deal',
    right_on='Company name',
    how='left'
)

# STEP 2: Select segmentation features
segmentation_cols = [
    'Company name', 'Industry_Standardized', 'Region', 'Revenue_Category',
    'Company_Size_Category', 'Technology_Count', 'Uses_Google_Tag_Manager',
    'Uses_Salesforce', 'Uses_Microsoft_Office_365', 'Annual Revenue',
    'Number of Employees', 'Amount', 'Forecast amount', 'Deal Score',
    'Deal probability', 'Pipeline_ticket', 'Ticket status', 'Training_Completion_Pct',
    'Implementation_Duration_Days', 'Time_To_Close_Hours'
]

categorical_cols = [
    'Industry_Standardized', 'Region', 'Revenue_Category',
    'Company_Size_Category', 'Pipeline_ticket', 'Ticket status'
]

# STEP 3: Drop missing values
segmentation_data = merged_full[segmentation_cols].dropna().copy()

# STEP 4: Encode categorical columns
segmentation_encoded = pd.get_dummies(segmentation_data, columns=categorical_cols, drop_first=True)

# STEP 5: Normalize numeric columns
numeric_cols = segmentation_encoded.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
segmentation_encoded[numeric_cols] = scaler.fit_transform(segmentation_encoded[numeric_cols])

# STEP 6: Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
segmentation_encoded['Cluster'] = kmeans.fit_predict(segmentation_encoded)

# STEP 7: Attach cluster back to original
segmentation_data['Cluster'] = segmentation_encoded['Cluster']

# STEP 8: Summary insights
summary = segmentation_data.groupby('Cluster')[['Annual Revenue', 'Amount', 'Deal Score']].mean().round(2)

# STEP 9: Plot average annual revenue per cluster
plt.figure(figsize=(8, 5))
sns.barplot(data=summary.reset_index(), x='Cluster', y='Annual Revenue')
plt.title('Average Annual Revenue by Cluster')
plt.ylabel('Annual Revenue')
plt.tight_layout()
plt.show()


KeyboardInterrupt: 